<a href="https://colab.research.google.com/github/EleonoraBartolomucci/Fairness/blob/master/Faceplusplus/merge_Scraping_Calrifai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#da cambiare:
#nome_ristorante
#nome_ristorante_staccato_sort con il sort (del file scraping)
#nome_ristorante_sort con il sort 
#users_restaurant_id (link condivisibile del csv con tutte le review)
#sentiment_id (link condivisibile del csv con il clarifai RANDOM)
#folder_id (cartella dove metto il file)

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
from google.colab import auth
import json
import pandas as pd
import csv

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

nome_ristorante = 'TupeloHoney'
nome_ristorante_staccato_sort = 'Tupelo Honey_date'
nome_ristorante_sort = 'TupeloHoney_Date'

# users_restaurant_id = id del file csv del ristorante (si trova in data > Scraping > Reviews_Relevance) > (nomeRistorante)_relevance/date_desc.csv)
#File dove ci sono id e recensione
users_restaurant_id = '15LwpZqBUvtbuePS-7K0ezRtfUecInjx9'  # FILE ID, got on google drive with condivision link
download = drive.CreateFile({'id': users_restaurant_id})
download.GetContentFile('%s' % nome_ristorante_staccato_sort + '_desc.csv')

#leggo il file csv del ristorante
sort = pd.read_csv('%s' % nome_ristorante_staccato_sort + '_desc.csv')

# sentiment_id = id del file csv del ristorante (si trova in data > face_data > Clarifai > (nomeRistorante) > (nomeRistorante)__NoDuplicati_Random_UsersData.csv)
clarifai_id = '14GZPcSjoaiGIKGaS-_XcHOz6rkGmg-7z'  # FILE ID, got on google drive with condivision link
download = drive.CreateFile({'id': clarifai_id})
download.GetContentFile('%s' % nome_ristorante + '_NoDuplicati_Random_UsersData.csv')

#leggo il file csv del ristorante
clarifai = pd.read_csv('%s' % nome_ristorante + '_NoDuplicati_Random_UsersData.csv')

#foto = pd.read_csv('%s' % nome_ristorante + '_FotoRandom.csv')
merged = sort.merge(clarifai,how = 'left',left_on='user_id', right_on='id')

review_random1 = merged.drop(columns=['id'])
review_random = review_random1.drop(columns=['text_review'])

#csv contenente user_id e review
review_random.to_csv('%s' % nome_ristorante_sort + '_UsersData.csv', index=False)

# making data frame from csv file 
nba = pd.read_csv('%s' % nome_ristorante_sort + '_UsersData.csv')

# replacing na values in college with No college 
nba["age"].fillna("Not recognized", inplace = True) 
nba["gender"].fillna("Not recognized", inplace = True) 
nba["ethnicity"].fillna("Not recognized", inplace = True) 

#csv contenente user_id e review
nba.to_csv('%s' % nome_ristorante_sort + '_UsersData.csv', index=False)

folder_id = '1IzKxj9S7BBipH2y832oGcAWPPQmxzTFs'    #cartella dove metto il csv
file_metadata = {'title': '%s' % nome_ristorante_sort + '_UsersData.csv',"parents":[{"id": folder_id,"kind":"drive#childList"}]}
folder = drive.CreateFile(file_metadata)
folder.SetContentFile('%s' % nome_ristorante_sort + '_UsersData.csv') #The contents of the file
folder.Upload()